In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("PySpark678 ") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1588280872526_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark_train=spark.read.format("csv").option("header", "true").load("s3://dataproject678/Train.csv").drop("_c0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark_test=spark.read.format("csv").option("header", "true").load("s3://dataproject678/Test.csv").drop("_c0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
for i in spark_train.columns:
    spark_train = spark_train.withColumn(i, spark_train[i].cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
for i in spark_test.columns:
    spark_test = spark_test.withColumn(i, spark_test[i].cast("double"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spark_train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_block_num: double (nullable = true)
 |-- shop_id: double (nullable = true)
 |-- item_id: double (nullable = true)
 |-- item_cnt_month: double (nullable = true)
 |-- item_price: double (nullable = true)
 |-- item_category_id: double (nullable = true)
 |-- item_cnt_month_lag_1: double (nullable = true)
 |-- item_cnt_month_lag_2: double (nullable = true)
 |-- item_cnt_month_lag_3: double (nullable = true)
 |-- item_cnt_month_lag_6: double (nullable = true)
 |-- item_cnt_month_lag_12: double (nullable = true)

In [8]:
spark_test.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+-------+--------------+----------+----------------+--------------------+--------------------+--------------------+--------------------+---------------------+
|date_block_num|shop_id|item_id|item_cnt_month|item_price|item_category_id|item_cnt_month_lag_1|item_cnt_month_lag_2|item_cnt_month_lag_3|item_cnt_month_lag_6|item_cnt_month_lag_12|
+--------------+-------+-------+--------------+----------+----------------+--------------------+--------------------+--------------------+--------------------+---------------------+
|          33.0|    2.0|   31.0|           1.0|     399.0|            37.0|                 0.0|                 0.0|                 0.0|                 0.0|                  0.0|
|          33.0|    2.0|  486.0|           3.0|     300.0|            73.0|                 1.0|                 0.0|                 0.0|                 1.0|                  1.0|
|          33.0|    2.0|  787.0|           1.0|     420.0|            49.0|               

In [79]:
shop_num=18
spark_train1=spark_train.filter(spark_train['shop_id'] <shop_num)
spark_test1=spark_test.filter(spark_test['shop_id'] <shop_num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
feature_list = []
for col in spark_train1.columns:
    if col == 'item_cnt_month':
        continue
    else:
        feature_list.append(col)
assembler = VectorAssembler(inputCols=feature_list,outputCol="features")
spark_train_v = assembler.transform(spark_train1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
spark_train_v2=spark_train_v.withColumnRenamed("item_cnt_month","label")
spark_train_v2=spark_train_v2[["features","label"]]
spark_train_v2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(10,[2,3,4],[32.0...|  6.0|
|(10,[2,3,4],[33.0...|  3.0|
|(10,[2,3,4],[35.0...|  1.0|
|(10,[2,3,4],[43.0...|  1.0|
|(10,[2,3,4],[51.0...|  2.0|
+--------------------+-----+
only showing top 5 rows

In [82]:
spark_test_v = assembler.transform(spark_test1)
spark_test_v2=spark_test_v.withColumnRenamed("item_cnt_month","label")
spark_test_v2=spark_test_v2[["features","label"]]
spark_test_v2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(10,[0,1,2,3,4],[...|  1.0|
|[33.0,2.0,486.0,3...|  3.0|
|(10,[0,1,2,3,4],[...|  1.0|
|(10,[0,1,2,3,4],[...|  1.0|
|(10,[0,1,2,3,4],[...|  1.0|
+--------------------+-----+
only showing top 5 rows

# Spark-Random Forest 

In [16]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
rf = RandomForestRegressor(labelCol='label', featuresCol='features',numTrees=80, maxDepth=8, seed=42)
model = rf.fit(spark_train_v2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
predictions = model.transform(spark_test_v2)
predictions.select("features","label", "prediction").show(5)
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|(10,[0,1,2,3,4],[...|  1.0|1.5221383572407008|
|[33.0,2.0,486.0,3...|  3.0|1.4769166459381713|
|(10,[0,1,2,3,4],[...|  1.0|1.3891930909393493|
|(10,[0,1,2,3,4],[...|  1.0|1.6170560174177653|
|(10,[0,1,2,3,4],[...|  1.0|1.2400418262074147|
+--------------------+-----+------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 2.34762

# Spark- Gradient Boosted Regression 

In [85]:
from pyspark.ml.regression import GBTRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
GBT= GBTRegressor(labelCol='label', featuresCol='features', maxDepth=6,maxIter=40)
model1 = GBT.fit(spark_train_v2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
predictions = model1.transform(spark_test_v2)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 2.04728